# Flexible Architecure with Optuna Optimization

In [ ]:
!pip install torchmetrics
!pip intall optuna

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

import torchmetrics
from torchmetrics import Accuracy, Precision, Recall, F1Score

import optuna

import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

# Dataset & DataLoaders

In [ ]:
def get_dataloaders(batch_size):

    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))
        
    ])

    val_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))
        
    ])

    train_dataset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=train_transform
    )
    
    val_dataset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=val_transform
    )

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    return train_loader, val_loader


# Model

In [ ]:
class FlexibleCNN(nn.Module):
    def __init__(self, n_layers, n_filters, kernel_sizes, dropout_rate, fc_size):
        super(FlexibleCNN, self).__init__()

        blocks = []

        in_channels = 3

        for i in range(n_layers):

            out_channels = n_filters[i]
            kernel_size = kernel_sizes[i]

            padding = (kernel_size - 1) // 2

            block = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )

            blocks.append(block)

            in_channels = out_channels
        
        self.features = nn.Sequential(*blocks)

        self.dropout_rate = dropout_rate
        self.fc_size = fc_size

        self.classifier = None
    
    def _create_classifier(self, flattened_size, device):
        self.classifier = nn.Sequential(
            nn.Dropout(self.dropout_rate),
            nn.Linear(flattened_size, self.fc_size),
            nn.ReLU(inplace=True),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.fc_size, 10)
        ).to(device)
    
    def forward(self, x):
        device = x.device

        x = self.features(x)

        flattened = torch.flatten(x, 1)
        flattened_size = flattened.size(1)

        if self.classifier is None:
            self._create_classifier(flattened_size, device)
        
        return self.classifier(flattened)

# Training & Validation Steps

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()

    total_loss = []
    loss_per_epoch = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backwards()
        optimizer.step()

        loss_per_epoch += loss.item()

    loss_per_epoch /= len(train_loader)
    total_loss.append(loss_per_epoch)

    return loss_per_epoch, total_loss